# Wind Turbine (Spark Demo)

Welcome! In this experiment we delve into the world of wind turbines
and harness the power of machine learning to predict their energy production!
In this demonstration, we will be using Spark to explore and augment the training
dataset and train a Gradient-Boosted Tree (GBT) regressor that will utilize various
features, such as wind speed and direction, to estimate the power output of a
wind turbine.

<div><img src="images/wind-farm.jpg" width="100%"/>
    <p>(Image generated by Stable Diffusion)</p>
</div>


Wind turbines hold tremendous potential as a sustainable source of energy,
capable of supplying a substantial portion of the world's power needs. However,
the inherent unpredictability of power generation poses a challenge when it
comes to optimizing this process.

Fortunately, we have a powerful tool at our disposal: machine learning. By
leveraging advanced algorithms and data analysis, we can develop models that
accurately predict the power production of wind turbines. This enables us to
optimize the power generation process and overcome the challenges associated
with its ingrained variability.

## Create a Spark Interactive Session

Let's begin! In this demo you'll be using Livy to create and manage an interactive
Spark session. Livy is an open-source REST service that enables remote and interactive
analytics on Apache Spark clusters. It provides a way to interact with Spark clusters
programmatically using a REST API, allowing you to submit Spark jobs, run interactive
queries, and manage Spark sessions.

First you need to connect to the Livy endpoint and create a new Spark interactive session.
This session will allow you to interact with Spark using your familiar notebook
environment, and execute Spark code to perform data processing tasks in an interactive manner.

The Spark interactive session is particularly useful for exploratory data analysis,
prototyping, and iterative development. It allows you to interactively work with large datasets,
perform transformations, apply analytical operations, and build machine learning models using
Spark's distributed computing capabilities. This is exactly what you'll do in this Notebook!

To communicate with Livy and manage your sessions you'll be using sparkmagic, an open-source
tool that provides a Jupyter kernel extension. Sparkmagic integrates with Livy, to provide
the underlying communication layer between the Jupyter kernel and the Spark cluster.

Execute the cell below and:

1. Select `Add Endpoint`
1. Select `Basic_Access`, paste your Livy endpoint and authenticate with your credentials
1. Select `Create Session`
1. Provide a name select `python` and click `Create Session`

When your session is ready the `Manage Sessions` pane will become active, providing you the session ID.
The session state will become `idle` which means that you are good to go!

> To configure Sparkmagic, you can make use of a config.json file located at ~/.sparkmagic/config.json.
> If you're running this notebook in a server that was created using the jupyter-data-science image,
> the default settings should suffice, and no additional configuration is required.

In [ ]:
%reload_ext sparkmagic.magics
%manage_spark

You are now prepared to embark on your first interaction with Livy.
Whenever you initiate a cell with the `%%spark` magic command, the code within
that cell will not be executed by your IPython kernel. Instead, it will be executed
within a Spark context managed by Livy. Rest assured, Livy will handle everything
seamlessly and provide you with a response containing the desired results.

Moreover, with Sparkmagic at your side, you can leave the networking part to it. Sparkmagic
takes care of all the intricate details, effortlessly creating and sending requests to
the Livy server. Finally, it seamlessly renders the response within the Jupyter user interface,
ensuring a smooth and hassle-free experience.

In [ ]:
%%spark
import pyspark
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from warnings import filterwarnings

from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark import SparkContext
from pyspark.sql.functions import substring
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import GBTRegressionModel
from pyspark.ml.evaluation import RegressionEvaluator
from py4j.java_gateway import java_import


sns.set_style('white')
filterwarnings('ignore')

In [ ]:
%%spark
spark = SparkSession.builder.appName("wind-turbine").getOrCreate()

In [ ]:
%%spark
java_import(spark._sc._jvm, "org.apache.spark.sql.api.python.*")

## Load the Dataset

It's time to load the dataset, which comes in a convenient CSV format.
You'll leverage the spark session you created to load it as a DataFrame. Once loaded,
you can delve into its contents by examining the first five rows and its schema.
Additionally, you'll get an idea of the dataset's size by printing the number of
examples available to us.

In [ ]:
%%spark
spark_df = spark.read.csv('file:///mounts/shared-volume/shared/spark/T1.csv', header=True, inferSchema=True)

Next, you should cache the dataset in memory. The `cache()` method is used to persist
or cache the contents of a DataFrame, Dataset, or RDD (Resilient Distributed Dataset)
in memory. Caching data in memory can significantly improve the performance of iterative
algorithms or repeated computations by avoiding the need to recompute or fetch the data
from disk.

In [ ]:
%%spark
# Caching the dataset
spark_df.cache()

# Converting all the column names to lower case
spark_df = spark_df.toDF(*[c.lower() for c in spark_df.columns])

print('Show the first 5 rows')
spark_df.show(5)

print('What are the variable data types?')
spark_df.printSchema()

print('How many observations do we have?')
spark_df.count()

In this experiment, the objective is to predict the power production of a wind turbine
(`lv activepower (kw)`) based on the dataset's other features. These features include
the current date and hour, the wind speed, and the wind direction. By analyzing the
relationships between these variables, you aim to uncover valuable insights and create
a predictive model that can estimate the power output of the turbine.

## Data exploration

Let's start exploring and transforming the dataset. First step, separate the `date/time`
column into two separate columns, one for the month and one for the hour of the day.

In [ ]:
%%spark
# Extracting a substring from columns to create month and hour variables
spark_df = spark_df.withColumn("month", substring("date/time", 4,2))
spark_df = spark_df.withColumn("hour", substring("date/time", 12,2))

# Converting string month and hour variables to integer
spark_df = spark_df.withColumn('month', spark_df.month.cast(IntegerType()))
spark_df = spark_df.withColumn('hour', spark_df.hour.cast(IntegerType()))

spark_df.show(5)

Moving forward, let's examine some essential statistical characteristics of our features,
specifically the mean and standard deviation. However, it's important to note that these statistics
are relevant only for the wind speed, theoretical power curve, and active power variables.

In [ ]:
%%spark
pd.options.display.float_format = '{:.2f}'.format
spark_df.select('wind speed (m/s)', 'theoretical_power_curve (kwh)', 'lv activepower (kw)').toPandas().describe()

Let's extract a random sample from the dataset and start creating a visual model.
By visualizing the data, we can gain a deeper understanding of its patterns, trends, and relationships.
Through this visual exploration, we'll uncover valuable insights that can guide us in our analysis
and decision-making processes.

In [ ]:
%%spark
# Taking a random sample from the big data
sample_df = spark_df.sample(withReplacement=False, fraction=0.1, seed=42).toPandas()

# Visualizing the distributions with the sample data
columns = ['wind speed (m/s)', 'wind direction (deg)', 'month', 'hour', 'theoretical_power_curve (kwh)', 'lv activepower (kw)']
i=1
plt.figure(figsize=(10, 12))
for each in columns:
    plt.subplot(3, 2, i)
    sample_df[each].plot.hist(bins=12)
    plt.title(each)
    i += 1
    
%matplot plt

In [ ]:
%%spark
plt.clf()

# Compute the average power production by month
monthly = spark_df.groupby('month').mean('lv activepower (kw)').sort('avg(lv activepower (kw))').toPandas()
sns.barplot(x='month', y='avg(lv activepower (kw))', data=monthly)
plt.title('Months and Average Power Production');

%matplot plt

In [ ]:
%%spark
plt.clf()

# Compute the average power production by hour
hourly = spark_df.groupby('hour').mean('lv activepower (kw)').sort('avg(lv activepower (kw))').toPandas()
sns.barplot(x='hour', y='avg(lv activepower (kw))', data=hourly)
plt.title('Hours and Average Power Production');

%matplot plt

In [ ]:
%%spark
# Compute the correlation between the features of the dataset
pd.set_option('display.max_columns', None)
sample_df[columns].corr()
plt.clf()
sns.pairplot(sample_df[columns], markers='*');
%matplot plt

In [ ]:
%%spark
# Finding the average power production for 5 m/s wind speed increments
avg_power = []
wind_speed = []

for i in [0, 5, 10, 15, 20]:
    avg_value = spark_df.filter((spark_df['wind speed (m/s)'] > i) 
                                & (spark_df['wind speed (m/s)'] <= i+5))\
                                .agg({'lv activepower (kw)':'mean'}).collect()[0][0] 
    avg_power.append(avg_value)
    wind_speed.append(str(i) + '-' + str(i+5))

plt.clf()
sns.barplot(x=wind_speed, y=avg_power, color='orange')
plt.title('Avg Power Production for 5 m/s Wind Speed Increments')
plt.xlabel('Wind Speed')
plt.ylabel('Average Power Production');

%matplot plt

In [ ]:
%%spark
# Creating the polar diagram
from math import radians

plt.clf()
plt.figure(figsize=(8,8))
ax = plt.subplot(111, polar=True)
# The circle position indicates wind direction
# The further from the circle center the higher the wind speed
# The circle size indicates active power
sns.scatterplot(x=[radians(x) for x in sample_df['wind direction (deg)']], 
                y=sample_df['wind speed (m/s)'],
                size=sample_df['lv activepower (kw)'],
                hue=sample_df['lv activepower (kw)'],
                alpha=0.7, legend=None)

# Setting the polar diagram's top to represent North 
ax.set_theta_zero_location('N')
# Setting -1 to start the wind direction clockwise
ax.set_theta_direction(-1)
# Setting wind speed labels in a better position to see
ax.set_rlabel_position(110)

plt.title('Wind Speed - Wind Direction - Power Production Diagram')
plt.ylabel(None);

%matplot plt

In [ ]:
%%spark
# Plot the real power production over the theoritical
plt.clf()
plt.figure(figsize=(10,6))
sns.scatterplot(x='wind speed (m/s)', y='lv activepower (kw)', color='orange', label='Real Production', alpha=0.5, data=sample_df)
sns.lineplot(x='wind speed (m/s)', y='theoretical_power_curve (kwh)', color='blue', label='Theoritical Production', data=sample_df)
plt.title('Wind Speed and Power Production Chart')
plt.ylabel('Power Production (kw)');

%matplot plt

In [ ]:
%%spark
# Filter the big data where the real and theoritical power productions are equal to 0
zero_theo_power = spark_df.filter((spark_df['lv activepower (kw)'] == 0)
                                  & (spark_df['theoretical_power_curve (kwh)'] == 0)).toPandas()

print(zero_theo_power[['wind speed (m/s)', 'theoretical_power_curve (kwh)', 'lv activepower (kw)']].sample(5))

In [ ]:
%%spark
plt.clf()
# Examine the wind speed distribution for 0 power production
zero_theo_power['wind speed (m/s)'].hist()
plt.title('Wind Speed Distribution for 0 Power Production')
plt.xlabel('Wind speed (m/s)')
plt.ylabel('Counts for 0 Power Production');

%matplot plt

In [ ]:
%%spark
# Observations for the wind speed > 3m/s and power production = 0, 
# while theoritically there should be power production
zero_power = spark_df.filter((spark_df['lv activepower (kw)'] == 0)
                            & (spark_df['theoretical_power_curve (kwh)'] != 0)
                            & (spark_df['wind speed (m/s)'] > 3)).toPandas()
print(zero_power.head())
print('No of Observations (while Wind Speed > 3 m/s and Power Production = 0): ', len(zero_power))

In [ ]:
%%spark
plt.clf()
zero_power['wind speed (m/s)'].plot.hist(bins=8)
plt.xlabel('Wind Speed (m/s)')
plt.ylabel('Counts for Zero Production')
plt.title('Wind Speed Counts for Zero Power Production')
plt.xticks(ticks=np.arange(4,18,2));

%matplot plt

In [ ]:
%%spark
# The number of the examples that show a 0 power production
# while theoritically there should be power production
print("The number of the examples that show a 0 power production"
      " while theoritically there should be power production:", zero_power['month'].count())

In [ ]:
%%spark
# Excluding the observations meeting the filter criterias 
spark_df = spark_df.filter(~((spark_df['lv activepower (kw)'] == 0)
                            & (spark_df['theoretical_power_curve (kwh)'] != 0)
                            & (spark_df['wind speed (m/s)'] > 3)))
spark_df.show(5)

In [ ]:
%%spark
columns = ['wind speed (m/s)', 'wind direction (deg)', 'theoretical_power_curve (kwh)', 'lv activepower (kw)']
i=1
plt.clf()
plt.figure(figsize=(20, 5))
for each in columns:
    df = spark_df.select(each).toPandas()
    plt.subplot(1, 4, i)
    #plt.boxplot(df)
    sns.boxplot(x=df[each])
    # plt.title(each)
    i += 1
    
%matplot plt

In [ ]:
%%spark
# Let's find and exclude possible outliers.
# Create a pandas df for visualization
wind_speed = spark_df.select('wind speed (m/s)').toPandas()

# Defining the quantiles and interquantile range
Q1 = wind_speed['wind speed (m/s)'].quantile(0.25)
Q3 = wind_speed['wind speed (m/s)'].quantile(0.75)
IQR = Q3-Q1
# Defining the lower and upper threshold values
lower = Q1 - 1.5*IQR
upper = Q3 + 1.5*IQR

print('Quantile (0.25): ', Q1, '  Quantile (0.75): ', Q3)
print('Lower threshold: ', lower, ' Upper threshold: ', upper)

In [ ]:
%%spark

# Fancy indexing for outliers
outlier_tf = (wind_speed['wind speed (m/s)'] < lower) | (wind_speed['wind speed (m/s)'] > upper)

print('Total Number of Outliers: ', len(wind_speed['wind speed (m/s)'][outlier_tf]))
print('--'*15)
print('Some Examples of Outliers:')
print(wind_speed['wind speed (m/s)'][outlier_tf].sample(10))

In [ ]:
%%spark
(spark_df.select('wind speed (m/s)', 'lv activepower (kw)')
         .filter(spark_df['wind speed (m/s)'] >= 19)
         .agg({'lv activepower (kw)':'mean'}).show())

In [ ]:
%%spark
from pyspark.sql import functions as F
spark_df = spark_df.withColumn('wind speed (m/s)', 
                               F.when(F.col('wind speed (m/s)') > 19.447, 19)
                               .otherwise(F.col('wind speed (m/s)')))
spark_df.count()

In [ ]:
%%spark
# High level power production
spark_df.filter(((spark_df['month'] == 3) | (spark_df['month'] == 8) | (spark_df['month'] == 11)) 
                & ((spark_df['hour'] >= 16) | (spark_df['hour'] <= 24)) 
                & ((spark_df['wind direction (deg)'] > 0) | (spark_df['wind direction (deg)'] < 90))
                & ((spark_df['wind direction (deg)'] > 180) | (spark_df['wind direction (deg)'] < 225))
               ).agg({'lv activepower (kw)':'mean'}).show()

In [ ]:
%%spark
# Low level power production
spark_df.filter((spark_df['month'] == 7) 
                & ((spark_df['hour'] >= 9) | (spark_df['hour'] <= 11)) 
                & ((spark_df['wind direction (deg)'] > 90) | (spark_df['wind direction (deg)'] < 160))
               ).agg({'lv activepower (kw)':'mean'}).show()

# Training a GBT Regressor

You are now ready to train our GBT regresson. To begin, you'll carefully specify
the features and the label for our model. Then, we'll split the dataset into training
and test subsets to ensure robust evaluation of our model's performance.
Finally, we'll initiate the training process, allowing our GBT regressor to learn
from the training data and make accurate predictions.

In [ ]:
%%spark
# Converting lv activepower (kw) variable as label
spark_df = spark_df.withColumn('label', spark_df['lv activepower (kw)'])

# Preparing the independent variables (Features) and fefining the variables to be used
variables = ['month', 'hour', 'wind speed (m/s)', 'wind direction (deg)']
vectorAssembler = VectorAssembler(inputCols = variables, outputCol = 'features')
va_df = vectorAssembler.transform(spark_df)

# Combining features and label column
final_df = va_df.select('features', 'label')
final_df.show(5)

In [ ]:
%%spark
# Split the dataset:
# 80% for training
# 20% for testing
splits = final_df.randomSplit([0.8, 0.2])
train_df = splits[0]
test_df = splits[1]

print('Train dataset: ', train_df.count())
print('Test dataset : ', test_df.count())

In [ ]:
%%spark
# Creating the gbm regressor object
gbm = GBTRegressor(featuresCol='features', labelCol='label')

# Training the model with train data
gbm_model = gbm.fit(train_df)

# Predicting using the test data
y_pred = gbm_model.transform(test_df)

# Initial look at the target and predicted values
y_pred.select('label', 'prediction').show(5)

In [ ]:
%%spark
gbm_model.write().overwrite().save("file:///mounts/shared-volume/user/spark/GBM.model")

In [ ]:
%%spark
gbm_model_dtap = GBTRegressionModel.load("file:///mounts/shared-volume/user/spark/GBM.model")

In [ ]:
%%spark
# Initial model success
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='label')

print('R2:\t', evaluator.evaluate(y_pred, {evaluator.metricName: 'r2'}))
print('MAE:\t', evaluator.evaluate(y_pred, {evaluator.metricName: 'mae'}))
print('RMSE:\t', evaluator.evaluate(y_pred, {evaluator.metricName: 'rmse'}))

In [ ]:
%%spark
# Converting sample_df back to Spark dataframe
eva_df = spark.createDataFrame(sample_df)

# Converting lv activepower (kw) variable as label
eva_df = eva_df.withColumn('label', eva_df['lv activepower (kw)'])

# Defining the variables to be used
variables = ['month', 'hour', 'wind speed (m/s)', 'wind direction (deg)']
vectorAssembler = VectorAssembler(inputCols = variables, outputCol = 'features')
vec_df = vectorAssembler.transform(eva_df)

# Combining features and label column
vec_df = vec_df.select('features', 'label')

# Using ML model to predict
preds = gbm_model.transform(vec_df)
preds_df = preds.select('label','prediction').toPandas()

# Compining dataframes to compare
frames = [sample_df[['wind speed (m/s)', 'theoretical_power_curve (kwh)']], preds_df]
sample_data = pd.concat(frames, axis=1)

plt.clf()
# Visualizing real, theoritical and predicted power production
plt.figure(figsize=(10, 7))
sns.scatterplot(x='wind speed (m/s)', y='label',alpha=0.5, label= 'Real Power', data=sample_data)
sns.scatterplot(x='wind speed (m/s)', y='prediction', alpha=0.7, label='Predicted Power', marker='o', data=sample_data)
sns.lineplot(x='wind speed (m/s)', y='theoretical_power_curve (kwh)', label='Theoritical Power',color='purple', data=sample_data)
plt.title('Wind Turbine Power Production Prediction')
plt.ylabel('Power Production (kw)')
plt.legend();

%matplot plt